In [2]:
from datasets import load_dataset, DatasetDict, load_from_disk
from transformers import AutoTokenizer, DataCollatorWithPadding

In [3]:
imdb = load_from_disk("../data/imdb")

In [4]:
imdb

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    dev: Dataset({
        features: ['text', 'label'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 24000
    })
})

In [5]:
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [7]:
max_sequence_length = 128
batch_size = 32
eval_steps = 100
learning_rate=2e-05
num_train_epochs=5
output_dir = "../output/"
model_dir = "../models/"
early_stopping_patience = 10

In [8]:
def tokenize_function(example):
    return tokenizer(example["text"],  truncation=True, padding="max_length", max_length=max_sequence_length )


tokenized_datasets = imdb.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/24 [00:00<?, ?ba/s]

In [9]:
tokenized_datasets = tokenized_datasets.remove_columns(["text", ])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")
tokenized_datasets["train"].column_names

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

In [10]:
# from torch.utils.data import DataLoader

# train_dataloader = DataLoader(
#     tokenized_datasets["train"], shuffle=True, batch_size=16, collate_fn=data_collator
# )
# dev_dataloader = DataLoader(
#     tokenized_datasets["dev"], batch_size=16, collate_fn=data_collator
# )

In [11]:
# for batch in train_dataloader:
#     break
# {k: v.shape for k, v in batch.items()}

In [12]:
from transformers import AutoModelForSequenceClassification, BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [13]:
from transformers import TrainingArguments
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import numpy as np

In [14]:
# def compute_metrics(eval_preds):
#     metric = load_metric("imdb)
#     logits, labels = eval_preds
#     predictions = np.argmax(logits, axis=-1)
#     return metric.compute(predictions=predictions, references=labels)

def compute_metrics(p):    
    pred, labels = p
    pred = np.argmax(pred, axis=1)
    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)    
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [15]:

# training_args = TrainingArguments('bert-base-uncased-imdb', 
#                                   learning_rate=2e-05,
#                                   per_device_train_batch_size =batch_size, 
#                                   per_device_eval_batch_size =batch_size,

#                           )

training_args = TrainingArguments(
   output_dir+"bert-base-uncased-imdb",
   evaluation_strategy ='steps',
   eval_steps = eval_steps , # Evaluation and Save happens every eval_steps steps
   save_total_limit = 1, # Only last  model is saved. Older ones are deleted.
   learning_rate=learning_rate,
   per_device_train_batch_size=batch_size,
   per_device_eval_batch_size=batch_size,
   num_train_epochs=num_train_epochs,
   metric_for_best_model = 'f1',
   load_best_model_at_end=True)

In [16]:
from transformers import Trainer, EarlyStoppingCallback

In [17]:

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["dev"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=early_stopping_patience)],
    compute_metrics=compute_metrics,
)


In [18]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1,Runtime,Samples Per Second
100,No log,0.371974,0.845000,0.798942,0.917004,0.853911,15.256900,65.544000
200,No log,0.349631,0.850000,0.873913,0.813765,0.842767,15.288600,65.408000
300,No log,0.359376,0.845000,0.869281,0.807692,0.837356,15.303900,65.343000
400,No log,0.307613,0.870000,0.851351,0.892713,0.871542,15.358400,65.111000
500,0.364500,0.309362,0.856000,0.807018,0.931174,0.864662,15.326100,65.248000
600,0.364500,0.302709,0.867000,0.881607,0.844130,0.862461,15.324400,65.255000
700,0.364500,0.300102,0.871000,0.894168,0.838057,0.865204,15.474900,64.621000
800,0.364500,0.383784,0.866000,0.833333,0.910931,0.870406,15.380100,65.019000
900,0.364500,0.309934,0.874000,0.881743,0.860324,0.870902,15.358900,65.109000
1000,0.254600,0.332236,0.872000,0.894397,0.840081,0.866388,15.442700,64.756000


TrainOutput(global_step=2700, training_loss=0.2004435383832013, metrics={'train_runtime': 4304.5331, 'train_samples_per_second': 0.908, 'total_flos': 7258763970957312, 'epoch': 3.45})

In [19]:
model.save_pretrained(model_dir+"bert-base-uncased-imdb")

In [20]:
predictions = trainer.predict(tokenized_datasets["test"])
print(predictions.predictions.shape, predictions.label_ids.shape)

(24000, 2) (24000,)


In [21]:
from sklearn.metrics import classification_report
preds = np.argmax(predictions.predictions, axis=-1)

In [22]:
print(classification_report(predictions.label_ids, preds))

              precision    recall  f1-score   support

           0       0.90      0.87      0.89     11994
           1       0.87      0.90      0.89     12006

    accuracy                           0.89     24000
   macro avg       0.89      0.89      0.89     24000
weighted avg       0.89      0.89      0.89     24000

